In [0]:
# Código para selecionar colunas que vou dar sequência na análise
# Fontes e bibliotecas
from pyspark.sql.functions import input_file_name
from pyspark.sql.types import *
from pyspark.sql.functions import col


# Parâmetros
s3_path = "s3a://data-architect-test-source/order.json.gz"
bronze_path = "dbfs:/mnt/bronze/raw_order"
s3_path_consumer = "s3a://data-architect-test-source/consumer.csv.gz"

# Leitura do JSON
df = spark.read.json(s3_path, multiLine=True)
df_consumer = spark.read.option("header", True).option("inferSchema", True).csv(s3_path_consumer)
df_teste_ab = spark.table('bronze.ab_test_ref')

# Seleção de colunas
colunas_selecionadas = ['customer_id', 'delivery_address_city', 'delivery_address_state' ,'order_id', 'merchant_id','order_scheduled','order_total_amount','origin_platform']

# Novo df
df_order_select = df.select(colunas_selecionadas)

# Gerando aliases
order = df_order_select.alias('order')
consumer = df_consumer.alias('consumer')
teste_ab = df_teste_ab.alias('test_ab')


# Join com a tabela consumer para trazer o status e com o teste ab para trazer o grupo.
df_final = order \
    .join(consumer, col('order.customer_id') == col('consumer.customer_id'), 'left') \
    .join(teste_ab, col('order.customer_id') == col('test_ab.customer_id'), 'left') \
    .select(
        col('order.*'),
        col('consumer.active'),
        col('test_ab.is_target')
        )
    
# Visualizando
#df_final.display()

# Salvando
spark.sql("CREATE DATABASE IF NOT EXISTS silver")
df_final.write.mode("overwrite").format("delta").saveAsTable("silver.orders_enriched")



In [0]:
# Código para verificar campos nulos
# Fontes e bibliotecas
from pyspark.sql.functions import input_file_name
from pyspark.sql.types import *
from pyspark.sql.functions import col


# Parâmetros
s3_path = "s3a://data-architect-test-source/order.json.gz"
bronze_path = "dbfs:/mnt/bronze/raw_order"
s3_path_consumer = "s3a://data-architect-test-source/consumer.csv.gz"

# Leitura do JSON
df = spark.read.json(s3_path, multiLine=True)
df_consumer = spark.read.option("header", True).option("inferSchema", True).csv(s3_path_consumer)
df_teste_ab = spark.table('bronze.ab_test_ref')

# Seleção de colunas
colunas_selecionadas = ['customer_id', 'delivery_address_city', 'delivery_address_state' ,'order_id', 'merchant_id','order_scheduled','order_total_amount','origin_platform']

# Novo df
df_order_select = df.select(colunas_selecionadas)

# Gerando aliases
order = df_order_select.alias('order')
consumer = df_consumer.alias('consumer')
teste_ab = df_teste_ab.alias('test_ab')


# Join com a tabela consumer para trazer o status e com o teste ab para trazer o grupo.
df_final = order \
    .join(consumer, col('order.customer_id') == col('consumer.customer_id'), 'left') \
    .join(teste_ab, col('order.customer_id') == col('test_ab.customer_id'), 'left') \
    .select(
        col('order.*'),
        col('consumer.active'),
        col('test_ab.is_target')
        )
    

from pyspark.sql.functions import countDistinct, count

# Contar apenas customer_id não nulos
non_null_customer_id = df_final.select(count("customer_id").alias("non_null_customer_id")).collect()[0]["non_null_customer_id"]
distinct_customer_id = df_final.select(countDistinct("customer_id").alias("distinct_customer_id")).collect()[0]["distinct_customer_id"]

# Contar apenas order_id não nulos
non_null_order_id = df_final.select(count("order_id").alias("non_null_order_id")).collect()[0]["non_null_order_id"]
distinct_order_id = df_final.select(countDistinct("order_id").alias("distinct_order_id")).collect()[0]["distinct_order_id"]

# Resultado - Temos 8.505 dados sem informação do customer_id, então não temos a informação da atividade e se participou do teste ab. Vou excluir
print(f"customer_id (não nulos): {non_null_customer_id}")
print(f"customer_id distintos (não nulos): {distinct_customer_id}")
print(f"order_id (não nulos): {non_null_order_id}")
print(f"order_id distintos (não nulos): {distinct_order_id}")


In [0]:
# Código para estudar na linha do tempo algumas variáveis como a quantidade de pedidos, valores, cidade, estado e tipo de plataforma
# Fontes e bibliotecas
from pyspark.sql.functions import input_file_name
from pyspark.sql.types import *
from pyspark.sql.functions import col, to_date, when, count, isnan, lit, to_timestamp
from pyspark.sql import SparkSession


# Parâmetros
s3_path = "s3a://data-architect-test-source/order.json.gz"
bronze_path = "dbfs:/mnt/bronze/raw_order"
s3_path_consumer = "s3a://data-architect-test-source/consumer.csv.gz"
s3_path_restaurant = "s3a://data-architect-test-source/restaurant.csv.gz"

# Leitura do JSON
df = spark.read.json(s3_path, multiLine=True)
df_consumer = spark.read.option("header", True).option("inferSchema", True).csv(s3_path_consumer)
df_restaurant = spark.read.option("header", True).option("inferSchema", True).csv(s3_path_restaurant)
df_teste_ab = spark.table('bronze.ab_test_ref')

# Seleção de colunas
colunas_selecionadas = ['customer_id', 'delivery_address_city', 'delivery_address_state' ,'order_id', 'merchant_id','order_created_at','order_total_amount','origin_platform']

# Novo df
df_order_select = df.select(colunas_selecionadas)

# Gerando aliases
order = df_order_select.alias('order')
consumer = df_consumer.alias('consumer')
restaurant = df_restaurant.alias('restaurant')
teste_ab = df_teste_ab.alias('test_ab')


# Join com a tabela consumer para trazer o status e com o teste ab para trazer o grupo.
df_final = order \
    .join(consumer, col('order.customer_id') == col('consumer.customer_id'), 'left') \
    .join(teste_ab, col('order.customer_id') == col('test_ab.customer_id'), 'left') \
    .join(restaurant, col('order.merchant_id') == col('restaurant.id'), 'left') \
    .select(
        col('order.*'),
        col('consumer.active'),
        col('test_ab.is_target'),
        col('restaurant.price_range'),
        col('restaurant.delivery_time'),
        col('restaurant.minimum_order_value')
        )
    
df_analise = df_final.withColumn("order_created_at", to_date(to_timestamp(col("order_created_at"))))

# Verificando mais nulidades
# encontrei mais 2 no campo origin_platform e 1.271 no campo active -> Vou seguir com a exclusão para limpar a base
#null_counts = df_final.select([
#    count(when(col(c).isNull(), c)).alias(c)
#    for c in df_final.columns
#])
#null_counts.show()

# Remoção dos nulos do campo customer_id, active e origin_platform
df_clean = df_analise.filter(
    df_analise["customer_id"].isNotNull() & df_analise['active'].isNotNull() & df_analise['origin_platform'].isNotNull() & df_analise['delivery_time'].isNotNull() & df_analise['minimum_order_value'].isNotNull()
)
    
# Visualizando
#df_final.display()

# Salvando
#spark.sql("CREATE DATABASE IF NOT EXISTS silver")
df_clean.write.mode("overwrite").format("delta").saveAsTable("silver.orders_time")

In [0]:
# Código para gerar a base sem as duplicações
from pyspark.sql.types import *
from pyspark.sql.functions import concat_ws, count

# Carregando a base
df = spark.table('silver.orders_time')


chaves = ['order_id','customer_id','order_total_amount','origin_platform']

# Criar o concat
df_chave = df.withColumn("chave_unica", concat_ws("|",*chaves))

# Remove as duplicações e também a coluna auxiliar                   
df_distinct = df_chave.dropDuplicates(['chave_unica']).drop('chave_unica')


# Salvando
df_distinct.write.mode("overwrite").format("delta").saveAsTable("silver.orders_time_distinct")

In [0]:
# Código para ajustar as variáveis delivery_time e minimum_order_value, transformando elas categorias
# regra delivery_time: < 19 sendo 1, >= 19 and < 45 sendo 2 e >= 45 sendo 3.
# regra minimum_order_value: < 15 sendo 1, >= 15 and < 21 sendo 2 e >= 21 sendo 3.
# Também vou excluir os dados com valores dos pedidos zerados e inserir a visão de região

# Código para gerar a base
from pyspark.sql.functions import when, col

# Carregando a base
df = spark.table('silver.orders_time_distinct')

# Manipulação das colunas
df_class = df \
    .withColumn(
    'delivery_time_class',
    when(col('delivery_time') < 19,1)
    .when((col('delivery_time') >= 19) & (col('delivery_time') < 45),2)
    .otherwise(3)
) \
    .withColumn(
    'minimum_order_value_class',
    when(col('minimum_order_value') < 15,1)
    .when((col('minimum_order_value') >= 15) & (col('minimum_order_value') < 21),2)
    .otherwise(3)
) \
    .withColumn(
    'delivery_region',
    when(col("delivery_address_state").isin("AC", "AP", "AM", "PA", "RO", "RR", "TO"), "Norte")
    .when(col("delivery_address_state").isin("AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"), "Nordeste")
    .when(col("delivery_address_state").isin("DF", "GO", "MT", "MS"), "Centro-Oeste")
    .when(col("delivery_address_state").isin("ES", "MG", "RJ", "SP"), "Sudeste")
    .when(col("delivery_address_state").isin("PR", "RS", "SC"), "Sul")
    .otherwise("Desconhecido")
    )

# Exclusão dos valores zerados do campo do valor do pedido
df_clean = df_class.filter(col('order_total_amount') != 0)

# Salvando
spark.sql("DROP TABLE IF EXISTS silver.orders_time_distinct_class")
df_clean.write.mode("overwrite").format("delta").saveAsTable("silver.orders_time_distinct_class")
